In [5]:
#import dependencies

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
from citipy import citipy 
import requests
from config import weather_api_key

In [6]:
#create a set of random latitudes and longitudes
lats = np.random.uniform(low = -90.000, high = 90.000, size = 2000)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [7]:
#Add longs and lats to a list

coordinates = list(lat_lngs)


In [8]:
#create cities list
cities = []
#Identify the nearest city for each latitude and longitude combination

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #add city if it is new to cities list
    if city not in cities:
        cities.append(city)

len(cities)


755

In [9]:
print(cities[1])

umarkot


In [3]:
#create an empty list to hold the wearther data
city_data = []
#create URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
#Print beginning of logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

#loop through all cities in the list

for i, city in enumerate(cities):
    # Group cities in sets of 50
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    city_url = url + "&q=" + city.replace(" ","+")
    
    #log URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # add 1 to the record count
    record_count += 1
    
    #Run an API request for each of the cities.
    
    try:
        #parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #Append the city information into city_data list.]
        city_data.append({"City": city.title(),
                      "Lat": city_lat,
                      "Lng": city_lng,
                      "Max Temp": city_max_temp,
                      "Humidity": city_humidity,
                      "Cloudiness": city_clouds,
                      "Wind Speed": city_wind,
                      "Country": city_country,
                      "Date": city_date,
                      "Current Description": city_weather_description})
    
    #If an error is experienced, skip the city
    
    except:
        print("City not found. Skipping...")
        pass
    
# #End data retrieval
# print("-------------------------")
# print("Data Retrieval Complete")

NameError: name 'weather_api_key' is not defined

In [4]:
city_weather_data_df = pd.DataFrame(city_data)
city_weather_data_df.drop(["Date"],axis = 1)

KeyError: "['Date'] not found in axis"

In [16]:
#reorder city data df columns
new_column_order = ["City", "Country","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_weather_data_df = city_weather_data_df[new_column_order]
city_weather_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Longjiang,CN,47.3303,123.1836,3.33,97,100,5.70,overcast clouds
1,Vaini,TO,-21.2000,-175.2000,82.56,83,40,10.36,scattered clouds
2,Muros,ES,42.7762,-9.0603,54.23,91,18,5.97,few clouds
3,Eilenburg,DE,51.4598,12.6334,49.24,68,100,9.01,overcast clouds
4,Cidreira,BR,-30.1811,-50.2056,72.39,69,29,11.88,scattered clouds
...,...,...,...,...,...,...,...,...,...
652,Leshukonskoye,RU,64.8989,45.7622,17.20,94,100,9.66,light snow
653,Chacabuco,AR,-34.6417,-60.4739,81.81,40,0,6.78,clear sky
654,Labuhan,ID,-6.8844,112.2051,77.34,89,100,9.10,overcast clouds
655,Kirksville,US,40.1948,-92.5833,47.05,49,40,14.97,scattered clouds


In [17]:
#create CSV file
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# export city_data into CSV
city_weather_data_df.to_csv(output_data_file, index_label = "City_ID")